In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd

from power_meter import *

# https://github.com/fzinfz/meter-dlt645
sys.path.append("/data/powermeter_dlt645/meter-dlt645")

from test_dlt645 import *
import test_dlt645

verbose=0

serial_port = '/dev/ttyUSB0'
timeout_count = 10
wait_for_read = 0.5

chn=dlt645.Channel(port_id = serial_port, tmo_cnt = timeout_count, wait_for_read = wait_for_read)

In [2]:
s = '''
000000004103 201  浩宁达DDZY22-Z
000081497010 5F   先行DDZY2188
000102866265 4F5F 正泰DDZY666
000012068388 401  杰瑞DDSF49
000000023557 402  宁光DDZY1122-Z
000002005433 405  华立DDZY285
'''

meters = [ re.findall('[^ ]+', line) for line in s.strip().splitlines() ]
df_meters = pd.DataFrame(meters, columns =['Addr','Tag','Model']).set_index('Addr')
df_meters

,Tag,Model
Addr,,
000000004103,201,浩宁达DDZY22-Z
000081497010,5F,先行DDZY2188
000102866265,4F5F,正泰DDZY666
000012068388,401,杰瑞DDSF49
000000023557,402,宁光DDZY1122-Z
000002005433,405,华立DDZY285


In [3]:
result = read_meters(chn, meters)


 ===== ['000000004103', '201', '浩宁达DDZY22-Z'] =====
{'A相电压': (230.8, 'V'),
 'A相电流': (0.155, 'A'),
 '内部电池电压': (3.6, 'V'),
 '功率-A相': ('35.77', 'W'),
 '功率-一分钟有功总平均': ('17.90', 'W'),
 '功率-瞬时总有功': ('17.50', 'W'),
 '功率因数-总': (0.474, ''),
 '日期时间': ('20201101 14:07:41', ''),
 '电能-组合有功总-上结算日': (2689.32, 'kWh'),
 '电能-组合有功总-当前': (2690.34, 'kWh'),
 '电能:本周期': ('1.02', 'kWh')}

 ===== ['000081497010', '5F', '先行DDZY2188'] =====
{'A相电压': (230.4, 'V'),
 'A相电流': (0.266, 'A'),
 '内部电池电压': (3.6, 'V'),
 '功率-A相': ('61.29', 'W'),
 '功率-一分钟有功总平均': ('61.30', 'W'),
 '功率-瞬时总有功': ('57.10', 'W'),
 '功率因数-总': (0.93, ''),
 '日期时间': ('20201101 14:03:54', ''),
 '电能-组合有功总-上结算日': (5242.71, 'kWh'),
 '电能-组合有功总-当前': (5439.07, 'kWh'),
 '电能:本周期': ('196.36', 'kWh')}

 ===== ['000102866265', '4F5F', '正泰DDZY666'] =====
{'A相电压': (230.6, 'V'),
 'A相电流': (7.367, 'A'),
 '内部电池电压': (3.65, 'V'),
 '功率-A相': ('1,698.83', 'W'),
 '功率-一分钟有功总平均': ('1,758.00', 'W'),
 '功率-瞬时总有功': ('1,701.10', 'W'),
 '功率因数-总': (1.0, ''),
 '日期时间': ('20201101 13:53:4

In [4]:
for k,v in result.items():
    v['Addr'] = k, ''
    
import copy
result_list = copy.deepcopy(result)

for m in result_list.values():
    for k in m.keys():
        m[k] = m[k][0]

df = pd.DataFrame(result_list.values()).set_index('Addr')
df = df_meters[['Tag']].merge(df, on='Addr')
df

,Tag,电能-组合有功总-当前,电能-组合有功总-上结算日,A相电压,A相电流,功率-瞬时总有功,功率-一分钟有功总平均,功率因数-总,内部电池电压,功率-A相,电能:本周期,日期时间
Addr,,,,,,,,,,,,
000000004103,201,2690.34,2689.32,230.8,0.155,17.50,17.90,0.474,3.60,35.77,1.02,20201101 14:07:41
000081497010,5F,5439.07,5242.71,230.4,0.266,57.10,61.30,0.930,3.60,61.29,196.36,20201101 14:03:54
000102866265,4F5F,19935.89,19928.71,230.6,7.367,"1,701.10","1,758.00",1.000,3.65,"1,698.83",7.18,20201101 13:53:42
000012068388,401,2248.70,2239.58,0.2,0.002,0.20,0.20,0.002,0.30,0.00,9.12,20201101 14:04:05
000000023557,402,619.86,618.90,230.9,0.022,0.00,0.20,0.000,3.59,5.08,0.96,20201101 14:05:40
000002005433,405,919.50,918.54,230.7,0.357,5.00,0.20,0.061,0.02,82.36,0.96,20201101 14:05:26
